In [ ]:
import glob
import os
import iris
from iris.time import PartialDateTime
from iris.util import equalise_attributes

DATADIR = '/project/ciid/projects/PRECIS/worksheets/data_afr22/AFR-22/'
# write outputs to own directory so that can be checked before upload under ciid
OUTDIR = f'{os.getenv("SCRATCH")}/cordex_inputs/'
retval = os.system('mkdir -p '+OUTDIR)

# Make data for worksheet 4

In [ ]:
# Prep future data (2041-2060) from ESGF CORDEX files
# Note this assumes that only want to process the REMO2015 data.

shortnames = {'MOHC-HadGEM2-ES':'hadgem2-es', 'MPI-M-MPI-ESM-LR':'mpi-esm-lr'}

# Make future 2021-2050 data
future_time_constraint = iris.Constraint(time=lambda cell: PartialDateTime(year=2041) 
                            <= cell.point <= PartialDateTime(year=2060))
for gcmid in ['MOHC-HadGEM2-ES', 'MPI-M-MPI-ESM-LR']:

    # set a short name consistent with what used in the exercises [would it be better to use full names?]
    shortgcm = shortnames[gcmid]
    for var in ['pr', 'tas']:

        # find all the files to consolidate 
        flist = glob.glob(DATADIR + f'{var}_*_{gcmid}_rcp85_*_GERICS-REMO2015_*_mon_*.nc')
        print (f'input files: {flist}')

        # merge the cubes
        datacubes = iris.load(flist, future_time_constraint)
        equalise_attributes(datacubes)
        cube = datacubes.concatenate_cube()
        
        if cube.name() == 'air_temperature':
            cube.convert_units('celsius')
            outpath = os.path.join(OUTDIR, 'future', shortgcm + '.mon.2041_2060.GERICS-REMO2015.tm.C.nc')
            iris.save(cube, outpath)
        if cube.name() == 'precipitation_flux':
            cube.convert_units('kg m-2 day-1')
            cube.units = 'mm day-1'
            outpath = os.path.join(OUTDIR, 'future', shortgcm + '.mon.2041_2060.GERICS-REMO2015.pr.mmday-1.nc')
            iris.save(cube, outpath)
        print(f'saved file: {outpath}')

In [ ]:
# Prep baseline data (1986-2005) from 
historical_time_constraint = iris.Constraint(time=lambda cell: PartialDateTime(year=1986) 
                            <= cell.point <= PartialDateTime(year=2005))
for gcmid in ['MOHC-HadGEM2-ES', 'MPI-M-MPI-ESM-LR']:

    # set a short name consistent with what used in the exercises [would it be better to use full names?]
    shortgcm = shortnames[gcmid]
    for var in ['pr', 'tas']:

        # find all the files to consolidate 
        flist = glob.glob(DATADIR + f'{var}_*_{gcmid}_historical_*_GERICS-REMO2015_*_mon_*.nc')
        flist = glob.glob(DATADIR + f'{var}_*_{gcmid}_historical_*_GERICS-REMO2015_*_mon_*.nc')
        print (f'input files: {flist}')

        # merge the cubes
        datacubes = iris.load(flist, historical_time_constraint)
        equalise_attributes(datacubes)
        cube = datacubes.concatenate_cube()
        
        if cube.name() == 'air_temperature':
            cube.convert_units('celsius')
            outpath = os.path.join(OUTDIR, 'historical', shortgcm + '.mon.1986_2005.GERICS-REMO2015.tm.C.nc')
            iris.save(cube, outpath)
        if cube.name() == 'precipitation_flux':
            cube.convert_units('kg m-2 day-1')
            cube.units = 'mm day-1'
            outpath = os.path.join(OUTDIR, 'historical', shortgcm + '.mon.1986_2005.GERICS-REMO2015.pr.mmday-1.nc')
            iris.save(cube, outpath)
        print(f'saved file: {outpath}')

# Make data for Worksheet 5

The files needed are daily precipitation, with the two periods (historical and future) consolidated into a single file

Note that these datasets should be saved with compression to reduce their file size.  If for some reason this needs to be done during a PRECIS course, it is quicker to save them without compression using `iris.save(cube, outfile)` but the filesize is significantly larger.

Unlike the previous PRECIS data, the memory requirements of this code are modest (probably due to the lack of rim removal meaning the data remains lazy). It does take several minutes (~5) to run each example.

In [ ]:
# Prep future data (2041-2060) from CORDEX files

shortnames = {'MOHC-HadGEM2-ES':'hadgem2-es', 'MPI-M-MPI-ESM-LR':'mpi-esm-lr'}

# Make future 2041-2060 data
future_time_constraint = iris.Constraint(time=lambda cell: PartialDateTime(year=2041) 
                            <= cell.point <= PartialDateTime(year=2060))
for gcmid in ['MOHC-HadGEM2-ES', 'MPI-M-MPI-ESM-LR']:

    # set a short name consistent with what used in the exercises [would it be better to use full names?]
    shortgcm = shortnames[gcmid]

    # find all the files 
    flist = glob.glob(DATADIR + f'pr_AFR-22_{gcmid}_rcp85_*_GERICS-REMO2015_*_day_*.nc')
    print (f'input files: {flist}')
    
    # load using time constraint above
    datacubes = iris.load(flist, future_time_constraint)

    # merge the cubes
    equalise_attributes(datacubes)
    cube = datacubes.concatenate_cube()
    
    if cube.name() == 'air_temperature':
        cube.convert_units('celsius')
        outpath = os.path.join(OUTDIR, 'future', shortgcm + '.day.2041_2060.GERICS-REMO2015.tm.C.nc')
        iris.save(cube, outpath, least_significant_digit=3, 
                  complevel=4, zlib=True)
    if cube.name() == 'precipitation_flux':
        cube.convert_units('kg m-2 day-1')
        cube.units = 'mm day-1'
        outpath = os.path.join(OUTDIR, 'future', shortgcm + '.day.2041_2060.GERICS-REMO2015.pr.mmday-1.nc')
        iris.save(cube, outpath, least_significant_digit=3, 
                  complevel=4, zlib=True)
    print(f'saved file: {outpath}')


In [ ]:
# Prep baseline data (1986-2006) from CORDEX files
historical_time_constraint = iris.Constraint(time=lambda cell: PartialDateTime(year=1986) 
                            <= cell.point <= PartialDateTime(year=2005))
# Make baseline data
for gcmid in ['MOHC-HadGEM2-ES', 'MPI-M-MPI-ESM-LR']:

    # set a short name consistent with what used in the exercises [would it be better to use full names?]
    shortgcm = shortnames[gcmid]

    # find all the files 
    flist = glob.glob(DATADIR + f'pr_AFR-22_{gcmid}_historical_*_GERICS-REMO2015_*_day_*.nc')
    print (f'input files: {flist}')
    
    # load using time constraint above
    datacubes = iris.load(flist, historical_time_constraint)

    # merge the cubes
    equalise_attributes(datacubes)
    cube = datacubes.concatenate_cube()
    
    if cube.name() == 'air_temperature':
        cube.convert_units('celsius')
        outpath = os.path.join(OUTDIR, 'historical', shortgcm + '.day.1986_2005.GERICS-REMO2015.tm.C.nc')
        iris.save(cube, outpath, least_significant_digit=3, 
                  complevel=4, zlib=True)
    if cube.name() == 'precipitation_flux':
        cube.convert_units('kg m-2 day-1')
        cube.units = 'mm day-1'
        outpath = os.path.join(OUTDIR, 'historical', shortgcm + '.day.1986_2005.GERICS-REMO2015.pr.mmday-1.nc')
        iris.save(cube, outpath, least_significant_digit=3, 
                  complevel=4, zlib=True)
    print(f'saved file: {outpath}')



<p><img src="img/MO_MASTER_black_mono_for_light_backg_RBG.png" alt="Met Office Logo" style="float: center; height: 100px;"/></p>
<center>© Crown Copyright, Met Office. All rights reserved.</center>
<center>This file is part of PyPrecis and is released under the BSD 3-Clause license.</center>
<center>See LICENSE in the root of the repository for full licensing details.</center>